<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Sistema de obtención de información con NLTK utilizando un corpus de wikipedia

In [1]:
# Importamos librerias

import string
import re # Regular Expressions (regex)
import nltk
import urllib.request
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\juani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\juani\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Datos
Se consumirán los datos del artículo de wikipedia sobre la historia Argentina

In [2]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Historia_de_la_Argentina')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text


In [3]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 243727


### 2 - Preprocesamiento
Definimos la funcion para hacer el preprocesamiento
- Sacar tildes de las palabra
- Remover caracteres especiales
- Quitar espacios o saltos
- Quitar caracteres de puntuación (opcional)
- Pasa a minúsculas todo el texto

In [4]:
def preprocess_clean_text(text, punctuation_removal = False):    
    # sacar tildes de las palabras:
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' # [^ : ningún caracter de todos estos
    # (termina eliminando cualquier caracter distinto de los del regex)
    text = re.sub(pattern, '', text)
    # substituir más de un caracter de espacio, salto de línea o tabulación
    text = re.sub(r'\s+', ' ', text)
    # quitar caracteres de puntuación
    if punctuation_removal:
        text = ''.join([c for c in text if c not in string.punctuation])
    # pasa a minúsculas todo el texto
    text = text.lower()
    return text

In [17]:
article = preprocess_clean_text(article_text)

In [6]:
print("Cantidad de caracteres en la nota:", len(article))

Cantidad de caracteres en la nota: 242415


### 3 - Dividir el texto en sentencias y en palabras

In [7]:
corpus = nltk.sent_tokenize(article) # divide en oraciones
words = nltk.word_tokenize(article) # divide en términos
print("corpus: " , len(corpus))
print("Vocabulario: ", len(words))

corpus:  1083
Vocabulario:  43206


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def get_processed_text(document):
    return nltk.word_tokenize(preprocess_clean_text(document, punctuation_removal = True))

# # ord() nos da el código Unicode para un caracter dado
# punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

# def get_processed_text(document):
#     # 1 - reduce el texto a mínuscula (string.lower())
#     # 2 - quitar los simbolos de puntuacion (string.translate())
#     # 3 - realiza la tokenización (nltk.word_tokenize)
#     # 4 - realiza la lematización (nuestra función perform_lemmatization)
#     return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

Defino los Stopwords en español

In [9]:
stopword = ['eramos', 'estabamos', 'estais', 'estan', 'estara', 'estaran', 'estaras', 'estare', 
 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 
 'estuvieramos', 'estuviesemos', 'fueramos', 'fuesemos', 'habeis', 'habia', 'habiais', 'habiamos', 'habian', 
 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 
 'hayais', 'hubieramos', 'hubiesemos', 'mas', 'mia', 'mias', 'mio', 'mios', 'seais', 'sera', 'seran', 
 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'si', 'tambien', 
 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 'tendriais', 'tendriamos', 'tendrian', 
  'tendrias', 'teneis', 'tengais', 'tenia', 'teniais', 'teniamos', 'tenian', 'tenias', 'tuvieramos', 'tuviesemos']

for w in stopwords.words('spanish'):
    preprocess_clean_text(w)
    stopword.append(w)
    

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [10]:


def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=stopword)

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    print('cosine similarity value: ', vector_matched)

    if vector_matched < 0.2: # si la similaridad coseno fue nula (ningún término en común)
        response = "Disculpame, no tengo esa respuesta"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
##### Pruebo los resultaods con varios inputs

In [11]:
user_input = "cuando fue la independencia"
generate_response(user_input, corpus)

c:\Users\juani\anaconda3\envs\tf\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


cosine similarity value:  0.31522622384213456


'en 1816 se declaro la independencia de las provincias unidas de america del sur en el congreso de tucuman.'

In [12]:
user_input = "decime algo sobre la revolución de mayo"
generate_response(user_input, corpus)

cosine similarity value:  0.3242510519923336


'el 25 de mayo de 1910 se cumplian 100 anos desde la revolucion de mayo, paso inicial de la independencia.'

In [13]:
user_input = "cual fue el decimo presidente"
generate_response(user_input, corpus)

cosine similarity value:  0.19318586788165126


'Disculpame, no tengo esa respuesta'

In [14]:
user_input = "quien gano la elección en 1916"
generate_response(user_input, corpus)

cosine similarity value:  0.1857284372122554


'Disculpame, no tengo esa respuesta'

In [15]:
user_input = "quien era el presidente en 1916"
generate_response(user_input, corpus)

cosine similarity value:  0.28477675879002606


'una vez que hipolito yrigoyen fue elegido presidente en 1916, comenzo a formarse dentro de la union civica radical un amplio sector que se opuso a yrigoyen, considerandolo autoritario.'

In [16]:
import gradio as gr

def bot_response(human_text):
    print("Q:", human_text)    
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

c:\Users\juani\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\juani\AppData\Local\Temp\ipykernel_8636\1968786774.py:9: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
